# Tutorial 3: Heat equation part I

## Introduction

In this tutorial we will start to learn how to use [`Gridap.jl`](https://github.com/gridap/Gridap.jl) for approximating transient PDEs by using time marching schemes (method of lines). We consider the *heat equation*, a.k.a. the transient Poisson equation.

$$
\left\lbrace
\begin{aligned}
\frac{\partial u(x,t)}{\partial t} -\kappa(x,t)\Delta u(t) = f(x,t)  \ &\text{in} \ \Omega,\\
u(t) = g_D(x,t) \ &\text{on}\ \Gamma_{\rm D},\\
u(0) = u_0(x) \ &\text{in}\ \Omega\\
\end{aligned}
\right.
$$

## Problem statement

In this tutorial we consider a rather simplified case to get started.

We want to solve the heat equation in a 2-dimensional domain $\Omega = [0, \pi]^2$ with
* homogenous Dirichlet boundaries on the whole boundary $\partial \Omega$
* homogeneous right-hand side $f=0$
* constant $\kappa  =  1$
* initial condition $u_0(x,y) = \sin(x) \sin(y)$

The weak form of the problem reads: find $u(t)\in U_g(t)$ such that

$$
m(t,u,v) + a(t,u,v) = b(t,v)\quad \forall v\in \ V
$$

Note that $U_g(t)$ is a transient FE space, in the sense that Dirichlet boundary value of functions in $U_g$ _can_ change in time (even though this is not the case in this tutorial). The definition of $m(u,v)$, $a(u,v)$ and $b(v)$ is as follows:

$$
\begin{aligned}
m(t,u,v) = \int_\Omega v\frac{\partial u}{\partial t} d\Omega, \\
a(t,u,v) = \int_\Omega \kappa(t) \nabla v\cdot \nabla u d\Omega, \\
b(t,v) = \int_\Omega v\ f(t) d\Omega
\end{aligned}
$$

## Discrete model and Triangulation

As usual, let us first load `Gridap`.

In [ ]:
using Gridap

First, we define the `DiscreteModel` and the `Triangulation`. More details on this can be found in [tutorial 2](https://gridap.github.io/Tutorials/stable/pages/t002_validation/).

In [ ]:
Nx, Ny = 10, 10
h = (2π)/Nx
print("Mesh size = $h")
𝒯 = CartesianDiscreteModel((0,2π,0,2π),(Nx,Ny)) |> simplexify
writevtk(𝒯,"model")

In [ ]:
# Exact solution
u(x,t::Real) = exp(-2*t)*sin(x[1])*sin(x[2])
u(t::Real) = x -> u(x,t)

# Heat conductivity
κ = 1

# Heat source
# Either compute it manually ...
f(x,t::Real) = 0.0
f(t::Real) = x -> f(x,t)
# Or use the inbuilt-capabilities
# f(t) = x -> ∂t(u)(x,t)-Δ(u(t))(x)
# or
# f(t) = x -> ∂t(u)(t)(x)-Δ(u(t))(x)

# Boundary data
g(x,t::Real) = 0.0
g(t::Real) = x -> g(x,t)

## Finite element spaces

In this tutorial we will use linear Lagrangian Finite Elements.

In [ ]:
order = 1
refFE = ReferenceFE(lagrangian,Float64,order)

The space of test functions is constant in time and is defined in steady problems:

In [ ]:
V = TestFESpace(𝒯,refFE,dirichlet_tags="boundary")

The trial space is now a `TransientTrialFESpace`, wich is constructed from a `TestFESpace` and a function (or vector of functions) for the Dirichlet boundary condition/s. In that case, the boundary condition function is a time-independent constant, but it could also be a time-dependent field depending on the coordinates $x$ and time $t$.

In [ ]:
U = TransientTrialFESpace(V,g)

## Weak form

As usual we build the following triangulation and the corresponding
Lebesgue measure, which will allow to write down integrals in a syntax
similar to the usual mathematical notation.

In [ ]:
Ω = Triangulation(𝒯)
degree = 2*order
dΩ = Measure(Ω,degree)

### FE operator definition

For time-dependent problems with constant coefficients one can use the optimized operator `TransientConstantMatrixFEOperator`, which assumes that the matrix contributions ($m$ and $a$) are time-independent. That is:

In [ ]:
m(u,v) = ∫( u*v )dΩ
a(u,v) = ∫(κ*(∇(u)⋅∇(v)))dΩ
b(v) = ∫( f(0.0)*v )dΩ
op = TransientConstantFEOperator(m,a,b,U,V)

## Transient solver

Once we have the FE operator defined, we proceed with the definition of the transient solver. First, we define a linear solver to be used at each time step. Here we use the `LUSolver`, but other choices are possible.

In [ ]:
linear_solver = LUSolver()

Then, we define the ODE solver. That is, the scheme that will be used
for the time integration. In this tutorial we use the `ThetaMethod`
with different $\theta$, in particular
* $\theta = 10^{-14} \approx 0$ corresponding to the *forward Euler method*, an unstable first order scheme,
* $\theta = 0.5$ corresponding to the *Crank-Nicolson*, a $A$-stable second order scheme,
* $\theta = 1.0$ corresponding to the *backward Euler*, a $A$-stable first order scheme.

We define the time step size
$\Delta t$ (constant) and the value of $\theta $.
The `ThetaMethod` function receives the linear solver, the time-step size $\Delta t$ and the $\theta$:

In [ ]:
# Backward Euler
θ = 1.0
Δt = 0.05
filesuffix="BE"

# Crank Nicolson
# θ = 0.5
# Δt = 0.05
# Δt = 0.05
# filesuffix="CN"

# Forward Euler
# θ = 1.0e-14
# Use parabolic CFL
# α = 0.1
# Δt = α*h^2
# filesuffix="FE"

ode_solver = ThetaMethod(linear_solver,Δt,θ)

Finally, we define the solution using the `solve` function, giving the ODE solver, the FE operator, an initial solution, an initial time and a final time. To construct the initial condition we interpolate the initial value (in that case a constant value of 0.0) into the FE space $U(t)$ at $t=0.0$.

In [ ]:
u₀ = interpolate_everywhere(u(0.0), U(0.0))
t₀ = 0.0
T = 1.0
uₕₜ = solve(ode_solver,op,u₀,t₀,T)

## Postprocessing

We should highlight that `uₕₜ` is just an _iterable_ function and the results at each time steps are only computed when iterating over it, i.e., lazily. We can post-process the results and generate the corresponding `vtk` files using the `createpvd` and `createvtk` functions. The former will create a `.pvd` file with the collection of `.vtu` files saved at each time step by `createvtk`. The computation of the problem solutions will be triggered in the following loop:

In [ ]:
createpvd("heat_equation_$(filesuffix)") do pvd
  for (uₕ,t) in uₕₜ
    pvd[t] = createvtk(Ω,"heat_equation_$(filesuffix)_$t"*".vtu",cellfields=["u"=>uₕ])
  end
end

![](../assets/poisson_transient/poisson_transient.gif)

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*